## 1. Data Preparation

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
from datetime import datetime
from dateutil import parser
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

!gdown 1Fqx7lZFI9DTP-uIV8gstnGoNVGygaeAO

with zipfile.ZipFile("../data/itineraries_csv.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/")
    


'gdown' is not recognized as an internal or external command,
operable program or batch file.


In [60]:

def load_data_into_dataframe(path):
    """
    Function to load data from all csv files into a dataframe

    Parameters
    ----------
    path : str
        Path to the root directory containing all the csv files
    """
    dfs_list = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                dfs_list.append(df)
        print('Finished loading files for: ' + str(root))
    final_df = pd.concat(dfs_list, ignore_index=True)
    return final_df

In [61]:
df_final = load_data_into_dataframe('../data/itineraries_csv')

Finished loading files for: ../data/itineraries_csv
Finished loading files for: ../data/itineraries_csv\ATL
Finished loading files for: ../data/itineraries_csv\BOS
Finished loading files for: ../data/itineraries_csv\CLT
Finished loading files for: ../data/itineraries_csv\DEN
Finished loading files for: ../data/itineraries_csv\DFW
Finished loading files for: ../data/itineraries_csv\DTW
Finished loading files for: ../data/itineraries_csv\EWR
Finished loading files for: ../data/itineraries_csv\IAD
Finished loading files for: ../data/itineraries_csv\JFK
Finished loading files for: ../data/itineraries_csv\LAX
Finished loading files for: ../data/itineraries_csv\LGA
Finished loading files for: ../data/itineraries_csv\MIA
Finished loading files for: ../data/itineraries_csv\OAK
Finished loading files for: ../data/itineraries_csv\ORD
Finished loading files for: ../data/itineraries_csv\PHL
Finished loading files for: ../data/itineraries_csv\SFO


In [62]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13519999 entries, 0 to 13519998
Data columns (total 23 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   legId                              object 
 1   searchDate                         object 
 2   flightDate                         object 
 3   startingAirport                    object 
 4   destinationAirport                 object 
 5   travelDuration                     object 
 6   isBasicEconomy                     bool   
 7   isRefundable                       bool   
 8   isNonStop                          bool   
 9   totalFare                          float64
 10  totalTravelDistance                float64
 11  segmentsDepartureTimeEpochSeconds  object 
 12  segmentsDepartureTimeRaw           object 
 13  segmentsArrivalTimeEpochSeconds    object 
 14  segmentsArrivalTimeRaw             object 
 15  segmentsArrivalAirportCode         object 
 16  segmentsDepartur

## 2. Feature Engineering

In [63]:
import re
## Building Feature Functions
def remove_utc_offset(datetime_str):
    """
    # Function to remove the UTC offset from datetime strings
    """
    dt = parser.parse(datetime_str)
    return dt.strftime('%Y-%m-%dT%H:%M:%S.%f')


def get_time_of_day(dt):
    """
    Function to generate time categories
    """
    hour = dt.hour
    if hour >=5 and hour < 8:
        return 'Early Morning'
    elif hour >= 8 and hour < 11:
        return 'Morning'
    elif hour >= 11 and hour < 14:
        return 'Midday'
    elif hour >= 14 and hour < 17:
        return 'Afternoon'
    elif hour >= 17 and hour < 20:
        return 'Evening'
    elif hour >= 20 and hour < 23:
        return 'Night'
    else:
        return 'Late Night'
    
# function to get features
def time_category_features(df):

    df['departuretime'] = df['segmentsDepartureTimeRaw'].apply(remove_utc_offset) 
    
    df['departuretime'] = pd.to_datetime(df['departuretime'], utc=False)
    
    # time category
    df['time_category'] = df['departuretime'].apply(get_time_of_day)
    
    # departure date
    df['date'] = (df['departuretime'] - pd.Timedelta(hours=2)).dt.date

    # no. of days from flight
    df['days_from_flight'] = (df['date'] - pd.to_datetime(df['searchDate']).dt.date)
      
    return df

def date_category_features(df):
    
    df["day_of_week"] = pd.to_datetime(df['date']).dt.day_name()
    
    df["year"] = pd.to_datetime(df['date']).dt.year
    
    df["month"] = pd.to_datetime(df['date']).dt.month
    
    df['day'] = pd.to_datetime(df['date']).dt.day
    
    return df


def extract_days(duration_string):
    days = re.search(r'(\d+) days', duration_string)
    return int(days.group(1)) if days else None


In [64]:
df_final = df_final[df_final['isNonStop'] == True]

In [186]:
df_input = df_final[['flightDate', 
                        'startingAirport',
                        'searchDate',
                        'destinationAirport', 
                        'segmentsDepartureTimeRaw', 
                        'segmentsCabinCode', 
                        'totalFare']]

df_input['segmentsDepartureTimeRaw'] = df_input['segmentsDepartureTimeRaw'].str.split('\|\|').str[0]
df_input['cabin_type'] = df_input['segmentsCabinCode'].str.split('\|\|').str[0]

In [187]:
df_input.isna().sum()

flightDate                  0
startingAirport             0
searchDate                  0
destinationAirport          0
segmentsDepartureTimeRaw    0
segmentsCabinCode           0
totalFare                   0
cabin_type                  0
dtype: int64

In [188]:
df_input = time_category_features(df= df_input)

In [189]:
df_input = date_category_features(df= df_input)

In [340]:
df = df_input.groupby(['startingAirport', 'destinationAirport', 'date', 'year', 'month', 'day', 'cabin_type', 'time_category', 'days_from_flight', 'day_of_week'])['totalFare'].mean().reset_index(name='mean_fare')

In [310]:
df

,startingAirport,destinationAirport,date,year,month,day,cabin_type,time_category,days_from_flight,day_of_week,mean_fare
0,ATL,BOS,2022-04-17,2022,4,17,coach,Afternoon,1 days,Sunday,398.600000
1,ATL,BOS,2022-04-17,2022,4,17,coach,Early Morning,1 days,Sunday,248.600000
2,ATL,BOS,2022-04-17,2022,4,17,coach,Evening,1 days,Sunday,387.956667
3,ATL,BOS,2022-04-17,2022,4,17,coach,Midday,1 days,Sunday,248.600000
4,ATL,BOS,2022-04-17,2022,4,17,coach,Morning,1 days,Sunday,265.766667
...,...,...,...,...,...,...,...,...,...,...,...
1580582,SFO,PHL,2022-07-16,2022,7,16,coach,Late Night,59 days,Saturday,468.610000
1580583,SFO,PHL,2022-07-16,2022,7,16,coach,Midday,59 days,Saturday,538.600000
1580584,SFO,PHL,2022-07-16,2022,7,16,coach,Night,59 days,Saturday,438.600000
1580585,SFO,PHL,2022-07-17,2022,7,17,coach,Midday,60 days,Sunday,548.600000


In [342]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1580587 entries, 0 to 1580586
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   startingAirport     1580587 non-null  object 
 1   destinationAirport  1580587 non-null  object 
 2   date                1580587 non-null  object 
 3   year                1580587 non-null  int32  
 4   month               1580587 non-null  int32  
 5   day                 1580587 non-null  int32  
 6   cabin_type          1580587 non-null  object 
 7   time_category       1580587 non-null  object 
 8   days_from_flight    1580587 non-null  object 
 9   day_of_week         1580587 non-null  object 
 10  mean_fare           1580587 non-null  float64
dtypes: float64(1), int32(3), object(7)
memory usage: 114.6+ MB


In [341]:
df['days_from_flight'] = df['days_from_flight'].astype('str')
#df['days_from_flight'] = df['days_from_flight'].apply(extract_days)

In [312]:
df.isna().sum()

startingAirport       0
destinationAirport    0
date                  0
year                  0
month                 0
day                   0
cabin_type            0
time_category         0
days_from_flight      0
day_of_week           0
mean_fare             0
dtype: int64

In [330]:
df['days_from_flight'].unique()

array(['1 days', '2 days', '3 days', '4 days', '5 days', '6 days',
       '7 days', '8 days', '9 days', '10 days', '11 days', '12 days',
       '13 days', '14 days', '15 days', '16 days', '17 days', '18 days',
       '19 days', '20 days', '21 days', '22 days', '23 days', '24 days',
       '25 days', '26 days', '27 days', '28 days', '29 days', '30 days',
       '31 days', '32 days', '33 days', '34 days', '35 days', '36 days',
       '37 days', '38 days', '39 days', '40 days', '41 days', '42 days',
       '43 days', '44 days', '45 days', '46 days', '47 days', '48 days',
       '49 days', '50 days', '51 days', '52 days', '53 days', '54 days',
       '55 days', '56 days', '57 days', '58 days', '59 days', '60 days',
       '0 days', '61 days'], dtype=object)

In [331]:
for col in df.columns:
  print(col)
  print(df[col].unique())

startingAirport
['ATL' 'BOS' 'CLT' 'DEN' 'DFW' 'DTW' 'EWR' 'IAD' 'JFK' 'LAX' 'LGA' 'MIA'
 'OAK' 'ORD' 'PHL' 'SFO']
destinationAirport
['BOS' 'CLT' 'DEN' 'DFW' 'DTW' 'EWR' 'IAD' 'JFK' 'LAX' 'LGA' 'MIA' 'ORD'
 'PHL' 'SFO' 'ATL' 'OAK']
date
<DatetimeArray>
['2022-04-17 00:00:00', '2022-04-18 00:00:00', '2022-04-19 00:00:00',
 '2022-04-20 00:00:00', '2022-04-21 00:00:00', '2022-04-22 00:00:00',
 '2022-04-23 00:00:00', '2022-04-24 00:00:00', '2022-04-25 00:00:00',
 '2022-04-26 00:00:00', '2022-04-27 00:00:00', '2022-04-28 00:00:00',
 '2022-04-29 00:00:00', '2022-04-30 00:00:00', '2022-05-01 00:00:00',
 '2022-05-02 00:00:00', '2022-05-03 00:00:00', '2022-05-04 00:00:00',
 '2022-05-05 00:00:00', '2022-05-06 00:00:00', '2022-05-07 00:00:00',
 '2022-05-08 00:00:00', '2022-05-09 00:00:00', '2022-05-10 00:00:00',
 '2022-05-11 00:00:00', '2022-05-12 00:00:00', '2022-05-13 00:00:00',
 '2022-05-14 00:00:00', '2022-05-15 00:00:00', '2022-05-16 00:00:00',
 '2022-05-17 00:00:00', '2022-05-18 00:00:00',

In [315]:
df['startingAirport'].unique()

array(['ATL', 'BOS', 'CLT', 'DEN', 'DFW', 'DTW', 'EWR', 'IAD', 'JFK',
       'LAX', 'LGA', 'MIA', 'OAK', 'ORD', 'PHL', 'SFO'], dtype=object)

In [343]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder

def create_transformer(cat_cols_1, cat_cols_2):
    cat_ohe_transformer = Pipeline(
        steps = [
            ('constant_imputer', SimpleImputer(strategy='constant', fill_value='MISSING')),
            ('one_hot_encoder', OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore'))
        ]
    )

    cat_target_transformer = Pipeline(
        steps = [
            ('constant_imputer', SimpleImputer(strategy='constant', fill_value='MISSING')),
            ('target_encoder', TargetEncoder())
        ]
    )
    
    preprocessor = ColumnTransformer(
        transformers = [
            ('low_cardinal_cats', cat_ohe_transformer, cat_cols_1),
            ('high_cardinal_cats', cat_target_transformer, cat_cols_2)
        ]
    )
    
    return preprocessor

In [344]:
cat_cols_2 = ['date', 'days_from_flight']
cat_cols_1 = ['startingAirport', 'destinationAirport', 'cabin_type', 
                    'time_category', 'day_of_week']

In [345]:
preprocessor = create_transformer(cat_cols_1= cat_cols_1, cat_cols_2= cat_cols_2)

## 3. Data Split

In [347]:
# Define the split date
df['date'] = pd.to_datetime(df['date'])
split_date = pd.to_datetime('2022-06-17')

# Create the train and test sets
df_train = df[df['date'] < split_date]
df_test = df[df['date'] >= split_date]

In [348]:
X_train = df_train.drop(['mean_fare', 'year', 'month', 'day'], axis=1)
y_train = df_train['mean_fare']
X_test = df_test.drop(['mean_fare', 'year', 'month', 'day'], axis=1)
y_test = df_test['mean_fare']

In [349]:
X_train['date'] = X_train['date'].astype(str)
X_test['date'] = X_test['date'].astype(str)

## 4. Train LightGBM Regressor Model

In [352]:
import lightgbm as lgb

def create_lgbm_pipeline(preprocessor, X_train, y_train, params):
    
    
    
    lgb_pipe = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('lgb', lgb.LGBMRegressor(**params))
        ]
    )
    
    lgb_pipe.fit(X_train, y_train)
    return lgb_pipe

In [354]:
default_parm = {}
lgb_default = create_lgbm_pipeline(preprocessor, X_train, y_train, default_parm)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018290 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 214
[LightGBM] [Info] Number of data points in the train set: 1266959, number of used features: 47
[LightGBM] [Info] Start training from score 295.985751


In [355]:
pred_1 = lgb_default.predict(X_test)

In [358]:
def calculate_regression_metrics(y_test, pred):
    rmse = mean_squared_error(y_test, pred, squared=False)
    mse = mean_squared_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    mae = mean_absolute_error(y_test, pred)
    metrics = {
        "RMSE": rmse,
        "MSE": mse,
        "R2": r2,
        "MAE": mae
    }
    return metrics

In [359]:
calculate_regression_metrics(y_test, pred_1)

{'RMSE': 102.06108554335506,
 'MSE': 10416.46518228804,
 'R2': 0.6117362648632906,
 'MAE': 70.431522794177}

In [360]:
def create_comparison_dataframe(y_test, predicted_values, model_name):
    df = pd.DataFrame({
        'Actual Value (y_test)': y_test,
        f'Predicted Value ({model_name})': predicted_values,
        'Difference': abs(y_test - predicted_values)
    })
    return df

In [361]:
df_pred = create_comparison_dataframe(y_test, pred_1, "LGBM_Default")


df_pred    

,Actual Value (y_test),Predicted Value (LGBM_Default),Difference
7362,233.605,264.136203,30.531203
7363,303.605,264.136203,39.468797
7364,288.610,262.848816,25.761184
7365,308.610,264.136203,44.473797
7366,288.610,260.395283,28.214717
...,...,...,...
1580582,468.610,387.134335,81.475665
1580583,538.600,659.560350,120.960350
1580584,438.600,467.469405,28.869405
1580585,548.600,665.464287,116.864287


## 5. Train tuned LGBM Regressor 

In [362]:
params = {
    'learning_rate': 0.1,
    'max_depth': 5,
    'n_estimators': 100,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_samples': 20,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1
}
lgb_pipe_tuned = create_lgbm_pipeline(preprocessor, X_train, y_train, params)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014938 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 214
[LightGBM] [Info] Number of data points in the train set: 1266959, number of used features: 47
[LightGBM] [Info] Start training from score 295.985751
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furt

In [363]:
pred_2 = lgb_pipe_tuned.predict(X_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [292]:
calculate_regression_metrics(y_test, pred_2)

{'RMSE': 103.14887175140247,
 'MSE': 10639.689743587272,
 'R2': 0.6034157837377294,
 'MAE': 71.8434317173128}

In [364]:
df_pred_2 = create_comparison_dataframe(y_test, pred_2, "LGBM_tuned")



df_pred_2    

,Actual Value (y_test),Predicted Value (LGBM_tuned),Difference
7362,233.605,265.454823,31.849823
7363,303.605,265.454823,38.150177
7364,288.610,262.084210,26.525790
7365,308.610,265.454823,43.155177
7366,288.610,257.104834,31.505166
...,...,...,...
1580582,468.610,421.808992,46.801008
1580583,538.600,637.354572,98.754572
1580584,438.600,498.405892,59.805892
1580585,548.600,641.995750,93.395750


## 6. Train Default XGBoost Model

In [370]:
import xgboost as xgb

def create_xgb_pipeline(preprocessor, X_train, y_train, params):
    xgb_pipe = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('xgb', xgb.XGBRegressor(**params))
        ]
    )
    xgb_pipe.fit(X_train, y_train)
    return xgb_pipe

In [371]:
default_parm = {}
xgb_default = create_xgb_pipeline(preprocessor, X_train, y_train, default_parm)

In [372]:
pred_3 = xgb_default.predict(X_test)

In [373]:
calculate_regression_metrics(y_test, pred_3)

{'RMSE': 102.65179553762525,
 'MSE': 10537.391127098417,
 'R2': 0.6072288664142613,
 'MAE': 70.662588873247}

In [374]:
df_pred_3 = create_comparison_dataframe(y_test, pred_3, "XGBOOST_Deafult")


df_pred_3    

,Actual Value (y_test),Predicted Value (XGBOOST_Deafult),Difference
7362,233.605,299.033356,65.428356
7363,303.605,299.033356,4.571644
7364,288.610,304.271179,15.661179
7365,308.610,299.033356,9.576644
7366,288.610,294.011536,5.401536
...,...,...,...
1580582,468.610,333.369904,135.240096
1580583,538.600,522.737488,15.862512
1580584,438.600,349.073853,89.526147
1580585,548.600,696.027405,147.427405


## 7. Tuned XGBoost Model

In [375]:
xgb_params = {
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'max_depth': 5,
    'min_child_weight': 3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'reg:squarederror',
    'n_jobs': -1,
    'booster': 'gbtree',
    'reg_alpha': 0.01,
    'reg_lambda': 1,
    'random_state': 42
}

xgb_pipe_tuned = create_xgb_pipeline(preprocessor, X_train, y_train, xgb_params)

In [376]:
pred_4 = xgb_pipe_tuned.predict(X_test)

In [377]:
calculate_regression_metrics(y_test, pred_4)

{'RMSE': 103.60015410760649,
 'MSE': 10732.991931119814,
 'R2': 0.5999380343098901,
 'MAE': 71.334566760529}

In [378]:
df_pred_4 = create_comparison_dataframe(y_test, pred_4, "XGBOOST_tuned")

df_pred_4    

,Actual Value (y_test),Predicted Value (XGBOOST_tuned),Difference
7362,233.605,278.222351,44.617351
7363,303.605,278.222351,25.382649
7364,288.610,278.169708,10.440292
7365,308.610,278.222351,30.387649
7366,288.610,272.959320,15.650680
...,...,...,...
1580582,468.610,337.750519,130.859481
1580583,538.600,586.744446,48.144446
1580584,438.600,409.643036,28.956964
1580585,548.600,701.679749,153.079749


In [379]:
import joblib 
joblib.dump(lgb_default, '../models/LGB_default.joblib')
joblib.dump(lgb_pipe_tuned, '../models/LGB_tuned.joblib')
joblib.dump(xgb_default, '../models/XGB_default.joblib')
joblib.dump(xgb_pipe_tuned, '../models/XGB_tuned.joblib')

['../models/XGB_tuned.joblib']